In [61]:
from functools import partial
from typing import List, Optional, Union

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float
from transformers import AutoModelForCausalLM

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

In [62]:
def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

In [63]:
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")

Disabled automatic differentiation


In [93]:
# NBVAL_IGNORE_OUTPUT
hf_model = AutoModelForCausalLM.from_pretrained("/mnt/hantao/models/gpt2_sft_2e5_3ep")
model = HookedTransformer.from_pretrained(
    "gpt2",
    hf_model=hf_model,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

# Get the default device used
device: torch.device = utils.get_device()

Loaded pretrained model gpt2 into HookedTransformer


In [95]:
example_prompt = "During their visit to the Billy, Jack and Ted decided to buy a cake for someone's birthday. Ted and Billy chose a chocolate cake to give to"
example_answer = " Jack"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'During', ' their', ' visit', ' to', ' the', ' Billy', ',', ' Jack', ' and', ' Ted', ' decided', ' to', ' buy', ' a', ' cake', ' for', ' someone', "'s", ' birthday', '.', ' Ted', ' and', ' Billy', ' chose', ' a', ' chocolate', ' cake', ' to', ' give', ' to']
Tokenized answer: [' Jack']


Performance on answer token:
Rank: 0        Logit: 19.11 Prob: 57.71% Token: | Jack|

Top 0th token. Logit: 19.11 Prob: 57.71% Token: | Jack|
Top 1th token. Logit: 18.03 Prob: 19.59% Token: | Ted|
Top 2th token. Logit: 17.29 Prob:  9.41% Token: | Billy|
Top 3th token. Logit: 16.41 Prob:  3.87% Token: | Bob|
Top 4th token. Logit: 16.28 Prob:  3.41% Token: | Jackie|
Top 5th token. Logit: 14.42 Prob:  0.53% Token: | Bobby|
Top 6th token. Logit: 14.18 Prob:  0.42% Token: | Sam|
Top 7th token. Logit: 13.78 Prob:  0.28% Token: | Charlie|
Top 8th token. Logit: 13.74 Prob:  0.27% Token: | Alice|
Top 9th token. Logit: 13.18 Prob:  0.15% Token: | John|


Ranks of the answer tokens: [(' Jack', 0)]

In [96]:
prompt_format = [
    "After John, Mary and Jim went to the grocery store and return, {} gave a bottle of milk to",
    "During the picnic, Billy, Jack, and Ted shared a blanket. {} passed a frisbee to",
    "After Sarah, Tom, and Alex went to the bookstore in the downtown, {} bought a new novel for",
    "Anna, Liam, and Noah decided to buy a cake for someone. {} chose a cake to give to",
]
names = [
    (" Mary", " John", " Jim"),
    (" Billy", " Jack", " Ted"),
    (" Sara", " Tom", " Alex"),
    (" Anna", " Liam", " Noah"),
]
# List of prompts
prompts = []
# List of answers, in the format (correct, incorrect)
answers = []
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = []
for i in range(len(prompt_format)):
    for j in range(3):
        answers.append((names[i][j], names[i][(j - 1)%3], names[i][(j - 2)%3]))
        answer_tokens.append(
            (
                model.to_single_token(answers[-1][0]),
                model.to_single_token(answers[-1][1]),
                model.to_single_token(answers[-1][2]),
            )
        )
        # Insert the *incorrect* answer to the prompt, making the correct answer the indirect object.
        prompts.append(prompt_format[i].format(answers[-1][1] + " and" + answers[-1][2]))
answer_tokens = torch.tensor(answer_tokens).to(device)
print(prompts)
print(answers)

['After John, Mary and Jim went to the grocery store and return,  Jim and John gave a bottle of milk to', 'After John, Mary and Jim went to the grocery store and return,  Mary and Jim gave a bottle of milk to', 'After John, Mary and Jim went to the grocery store and return,  John and Mary gave a bottle of milk to', 'During the picnic, Billy, Jack, and Ted shared a blanket.  Ted and Jack passed a frisbee to', 'During the picnic, Billy, Jack, and Ted shared a blanket.  Billy and Ted passed a frisbee to', 'During the picnic, Billy, Jack, and Ted shared a blanket.  Jack and Billy passed a frisbee to', 'After Sarah, Tom, and Alex went to the bookstore in the downtown,  Alex and Tom bought a new novel for', 'After Sarah, Tom, and Alex went to the bookstore in the downtown,  Sara and Alex bought a new novel for', 'After Sarah, Tom, and Alex went to the bookstore in the downtown,  Tom and Sara bought a new novel for', 'Anna, Liam, and Noah decided to buy a cake for someone.  Noah and Liam chos

In [97]:
for prompt in prompts:
    str_tokens = model.to_str_tokens(prompt)
    print("Prompt length:", len(str_tokens))
    print("Prompt as tokens:", str_tokens)

Prompt length: 25
Prompt as tokens: ['<|endoftext|>', 'After', ' John', ',', ' Mary', ' and', ' Jim', ' went', ' to', ' the', ' grocery', ' store', ' and', ' return', ',', ' ', ' Jim', ' and', ' John', ' gave', ' a', ' bottle', ' of', ' milk', ' to']
Prompt length: 25
Prompt as tokens: ['<|endoftext|>', 'After', ' John', ',', ' Mary', ' and', ' Jim', ' went', ' to', ' the', ' grocery', ' store', ' and', ' return', ',', ' ', ' Mary', ' and', ' Jim', ' gave', ' a', ' bottle', ' of', ' milk', ' to']
Prompt length: 25
Prompt as tokens: ['<|endoftext|>', 'After', ' John', ',', ' Mary', ' and', ' Jim', ' went', ' to', ' the', ' grocery', ' store', ' and', ' return', ',', ' ', ' John', ' and', ' Mary', ' gave', ' a', ' bottle', ' of', ' milk', ' to']
Prompt length: 25
Prompt as tokens: ['<|endoftext|>', 'During', ' the', ' picnic', ',', ' Billy', ',', ' Jack', ',', ' and', ' Ted', ' shared', ' a', ' blanket', '.', ' ', ' Ted', ' and', ' Jack', ' passed', ' a', ' fr', 'is', 'bee', ' to']
Promp

In [98]:
tokens = model.to_tokens(prompts, prepend_bos=True)

# Run the model and cache all activations
original_logits, cache = model.run_with_cache(tokens)

In [99]:
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()


print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(original_logits, answer_tokens).item(), 3),
)

Per prompt logit difference: tensor([ 2.3410,  1.9170,  3.9460,  3.2890,  1.0880,  1.9460, -6.8480,  4.4840,
         4.4410,  5.2270,  3.9910,  0.9180])
Average logit difference: 2.228


In [100]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
print("Answer residual directions shape:", answer_residual_directions.shape)
logit_diff_directions = (
    answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
)
print("Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([12, 3, 768])
Logit difference directions shape: torch.Size([12, 768])


In [101]:
# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type].
final_residual_stream = cache["resid_post", -1]
print("Final residual stream shape:", final_residual_stream.shape)
final_token_residual_stream = final_residual_stream[:, -1, :]
# Apply LayerNorm scaling
# pos_slice is the subset of the positions we take - here the final token of each prompt
scaled_final_token_residual_stream = cache.apply_ln_to_stack(
    final_token_residual_stream, layer=-1, pos_slice=-1
)

average_logit_diff = einsum(
    "batch d_model, batch d_model -> ",
    scaled_final_token_residual_stream,
    logit_diff_directions,
) / len(prompts)
print("Calculated average logit diff:", round(average_logit_diff.item(), 3))
print("Original logit difference:", round(original_average_logit_diff.item(), 3))

Final residual stream shape: torch.Size([12, 25, 768])
Calculated average logit diff: 2.228
Original logit difference: 2.228


In [102]:
def residual_stack_to_logit_diff(
    residual_stack: Float[torch.Tensor, "components batch d_model"],
    cache: ActivationCache,
) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(
        residual_stack, layer=-1, pos_slice=-1
    )
    return einsum(
        "... batch d_model, batch d_model -> ...",
        scaled_residual_stack,
        logit_diff_directions,
    ) / len(prompts)

In [103]:
accumulated_residual, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True
)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, cache)
line(
    logit_lens_logit_diffs,
    x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
    hover_name=labels,
    title="Logit Difference From Accumulate Residual Stream",
)

In [104]:
per_layer_residual, labels = cache.decompose_resid(
    layer=-1, pos_slice=-1, return_labels=True
)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

Compared with the logit difference of the original model, these two plots show that most parts of the process is quite the same, although at layer 10 is displays some difference: its logit difference continues to grow(although decreases at 11 remains the same) instead of beginning to decrease.
That suggestst that the difference might happens in layer 10.
Let's make a gif graph of the sliced models.

In [105]:
per_head_residual, labels = cache.stack_head_results(
    layer=-1, pos_slice=-1, return_labels=True
)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache)
per_head_logit_diffs = einops.rearrange(
    per_head_logit_diffs,
    "(layer head_index) -> layer head_index",
    layer=model.cfg.n_layers,
    head_index=model.cfg.n_heads,
)
imshow(
    per_head_logit_diffs,
    labels={"x": "Head", "y": "Layer"},
    title="Logit Difference From Each Head",
)

Tried to stack head results when they weren't cached. Computing head results now


L10H0 and L10H7 calls for my attention. its activation rate has the biggest changing over the time. 

In [106]:
def visualize_attention_patterns(
    heads: Union[List[int], int, Float[torch.Tensor, "heads"]],
    local_cache: ActivationCache,
    local_tokens: torch.Tensor,
    title: Optional[str] = "",
    max_width: Optional[int] = 700,
) -> str:
    # If a single head is given, convert to a list
    if isinstance(heads, int):
        heads = [heads]

    # Create the plotting data
    labels: List[str] = []
    patterns: List[Float[torch.Tensor, "dest_pos src_pos"]] = []

    # Assume we have a single batch item
    batch_index = 0

    for head in heads:
        # Set the label
        layer = head // model.cfg.n_heads
        head_index = head % model.cfg.n_heads
        labels.append(f"L{layer}H{head_index}")

        # Get the attention patterns for the head
        # Attention patterns have shape [batch, head_index, query_pos, key_pos]
        patterns.append(local_cache["attn", layer][batch_index, head_index])

    # Convert the tokens to strings (for the axis labels)
    str_tokens = model.to_str_tokens(local_tokens)

    # Combine the patterns into a single tensor
    patterns: Float[torch.Tensor, "head_index dest_pos src_pos"] = torch.stack(
        patterns, dim=0
    )

    # Circuitsvis Plot (note we get the code version so we can concatenate with the title)
    plot = attention_heads(
        attention=patterns, tokens=str_tokens, attention_head_names=labels
    ).show_code()

    # Display the title
    title_html = f"<h2>{title}</h2><br/>"

    # Return the visualisation as raw code
    return f"<div style='max-width: {str(max_width)}px;'>{title_html + plot}</div>"

In [107]:
top_k = 3

top_positive_logit_attr_heads = torch.topk(
    per_head_logit_diffs.flatten(), k=top_k
).indices

positive_html = visualize_attention_patterns(
    top_positive_logit_attr_heads,
    cache,
    tokens[0],
    f"Top {top_k} Positive Logit Attribution Heads",
)

top_negative_logit_attr_heads = torch.topk(
    -per_head_logit_diffs.flatten(), k=top_k
).indices

negative_html = visualize_attention_patterns(
    top_negative_logit_attr_heads,
    cache,
    tokens[0],
    title=f"Top {top_k} Negative Logit Attribution Heads",
)

HTML(positive_html + negative_html)

In [133]:
corrupted_prompts = []
for i in range(0, len(prompts), 3):
    # Add second prompt in the trio first
    corrupted_prompts.append(prompts[(i + 1) % len(prompts)])
    # Then add the first prompt
    corrupted_prompts.append(prompts[i % len(prompts)])
    # Finally, add the third prompt, wrapping around the list safely
    corrupted_prompts.append(prompts[(i + 2) % len(prompts)])
    

# Proceed with tokenization and model running as before
corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")
corrupted_average_logit_diff = logits_to_ave_logit_diff(corrupted_logits, answer_tokens)
print("Corrupted Average Logit Diff:", round(corrupted_average_logit_diff.item(), 2))
print("Clean Average Logit Diff:", round(original_average_logit_diff.item(), 2))

Corrupted Average Logit Diff: 0.14
Clean Average Logit Diff: 2.23


In [134]:
model.to_string(corrupted_tokens)

['<|endoftext|>After John, Mary and Jim went to the grocery store and return,  Mary and Jim gave a bottle of milk to',
 '<|endoftext|>After John, Mary and Jim went to the grocery store and return,  Jim and John gave a bottle of milk to',
 '<|endoftext|>After John, Mary and Jim went to the grocery store and return,  John and Mary gave a bottle of milk to',
 '<|endoftext|>During the picnic, Billy, Jack, and Ted shared a blanket.  Billy and Ted passed a frisbee to',
 '<|endoftext|>During the picnic, Billy, Jack, and Ted shared a blanket.  Ted and Jack passed a frisbee to',
 '<|endoftext|>During the picnic, Billy, Jack, and Ted shared a blanket.  Jack and Billy passed a frisbee to',
 '<|endoftext|>After Sarah, Tom, and Alex went to the bookstore in the downtown,  Sara and Alex bought a new novel for',
 '<|endoftext|>After Sarah, Tom, and Alex went to the bookstore in the downtown,  Alex and Tom bought a new novel for',
 '<|endoftext|>After Sarah, Tom, and Alex went to the bookstore in the 

In [135]:
def patch_residual_component(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    pos,
    clean_cache,
):
    corrupted_residual_component[:, pos, :] = clean_cache[hook.name][:, pos, :]
    return corrupted_residual_component


def normalize_patched_logit_diff(patched_logit_diff):
    # Subtract corrupted logit diff to measure the improvement, divide by the total improvement from clean to corrupted to normalise
    # 0 means zero change, negative means actively made worse, 1 means totally recovered clean performance, >1 means actively *improved* on clean performance
    return (patched_logit_diff - corrupted_average_logit_diff) / (
        original_average_logit_diff - corrupted_average_logit_diff
    )


patched_residual_stream_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("resid_pre", layer), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_residual_stream_diff[layer, position] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [136]:
prompt_position_labels = [
    f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(tokens[0]))
]
imshow(
    patched_residual_stream_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched Residual Stream",
    labels={"x": "Position", "y": "Layer"},
)

The info of "Jim", "and", "John"(John is mild, but exists) is moved to the last token in the 7/8 layers. The difference is that "and" and a little bit of "John" is moved toward the last token too. Let's make a gif out of that.

In [112]:
patched_attn_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
patched_mlp_diff = torch.zeros(
    model.cfg.n_layers, tokens.shape[1], device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for position in range(tokens.shape[1]):
        hook_fn = partial(patch_residual_component, pos=position, clean_cache=cache)
        patched_attn_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("attn_out", layer), hook_fn)],
            return_type="logits",
        )
        patched_attn_logit_diff = logits_to_ave_logit_diff(
            patched_attn_logits, answer_tokens
        )
        patched_mlp_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("mlp_out", layer), hook_fn)],
            return_type="logits",
        )
        patched_mlp_logit_diff = logits_to_ave_logit_diff(
            patched_mlp_logits, answer_tokens
        )

        patched_attn_diff[layer, position] = normalize_patched_logit_diff(
            patched_attn_logit_diff
        )
        patched_mlp_diff[layer, position] = normalize_patched_logit_diff(
            patched_mlp_logit_diff
        )

In [113]:
imshow(
    patched_attn_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched Attention Layer",
    labels={"x": "Position", "y": "Layer"},
)

In [114]:
imshow(
    patched_mlp_diff,
    x=prompt_position_labels,
    title="Logit Difference From Patched MLP Layer",
    labels={"x": "Position", "y": "Layer"},
)

In [115]:
def patch_head_vector(
    corrupted_head_vector: Float[torch.Tensor, "batch pos head_index d_head"],
    hook,
    head_index,
    clean_cache,
):
    corrupted_head_vector[:, :, head_index, :] = clean_cache[hook.name][
        :, :, head_index, :
    ]
    return corrupted_head_vector


patched_head_z_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("z", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_z_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [116]:
imshow(
    patched_head_z_diff,
    title="Logit Difference From Patched Head Output",
    labels={"x": "Head", "y": "Layer"},
)

In [117]:
patched_head_v_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_vector, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("v", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_v_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [118]:
imshow(
    patched_head_v_diff,
    title="Logit Difference From Patched Head Value",
    labels={"x": "Head", "y": "Layer"},
)

In [119]:
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(patched_head_v_diff.flatten()),
    y=utils.to_numpy(patched_head_z_diff.flatten()),
    xaxis="Value Patch",
    yaxis="Output Patch",
    caxis="Layer",
    hover_name=head_labels,
    color=einops.repeat(
        np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads
    ),
    range_x=(-0.5, 0.5),
    range_y=(-0.5, 0.5),
    title="Scatter plot of output patching vs value patching",
)

In [120]:
def patch_head_pattern(
    corrupted_head_pattern: Float[torch.Tensor, "batch head_index query_pos d_head"],
    hook,
    head_index,
    clean_cache,
):
    corrupted_head_pattern[:, head_index, :, :] = clean_cache[hook.name][
        :, head_index, :, :
    ]
    return corrupted_head_pattern


patched_head_attn_diff = torch.zeros(
    model.cfg.n_layers, model.cfg.n_heads, device=device, dtype=torch.float32
)
for layer in range(model.cfg.n_layers):
    for head_index in range(model.cfg.n_heads):
        hook_fn = partial(patch_head_pattern, head_index=head_index, clean_cache=cache)
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(utils.get_act_name("attn", layer, "attn"), hook_fn)],
            return_type="logits",
        )
        patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

        patched_head_attn_diff[layer, head_index] = normalize_patched_logit_diff(
            patched_logit_diff
        )

In [121]:
imshow(
    patched_head_attn_diff,
    title="Logit Difference From Patched Head Pattern",
    labels={"x": "Head", "y": "Layer"},
)
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(patched_head_attn_diff.flatten()),
    y=utils.to_numpy(patched_head_z_diff.flatten()),
    hover_name=head_labels,
    xaxis="Attention Patch",
    yaxis="Output Patch",
    title="Scatter plot of output patching vs attention patching",
)

In [122]:
top_k = 10
top_heads_by_output_patch = torch.topk(
    patched_head_z_diff.abs().flatten(), k=top_k
).indices
first_mid_layer = 7
first_late_layer = 9
early_heads = top_heads_by_output_patch[
    top_heads_by_output_patch < model.cfg.n_heads * first_mid_layer
]
mid_heads = top_heads_by_output_patch[
    torch.logical_and(
        model.cfg.n_heads * first_mid_layer <= top_heads_by_output_patch,
        top_heads_by_output_patch < model.cfg.n_heads * first_late_layer,
    )
]
late_heads = top_heads_by_output_patch[
    model.cfg.n_heads * first_late_layer <= top_heads_by_output_patch
]

early = visualize_attention_patterns(
    early_heads, cache, tokens[0], title=f"Top Early Heads"
)
mid = visualize_attention_patterns(
    mid_heads, cache, tokens[0], title=f"Top Middle Heads"
)
late = visualize_attention_patterns(
    late_heads, cache, tokens[0], title=f"Top Late Heads"
)

HTML(early + mid + late)